# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the libraries required.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks  = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2527438894517,
 'week52high': 160.51,
 'week52low': 114.4,
 'week52highSplitAdjustOnly': 157.75,
 'week52lowSplitAdjustOnly': 116.38,
 'week52change': 0.247725845366041,
 'sharesOutstanding': 17107130782,
 'float': 0,
 'avg10Volume': 60952694,
 'avg30Volume': 72450433,
 'day200MovingAvg': 145.36,
 'day50MovingAvg': 149.3,
 'employees': 150421,
 'ttmEPS': 11.33,
 'ttmDividendRate': 0.8760423551820117,
 'dividendYield': 0.005953492707865629,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-29',
 'nextEarningsDate': '2022-01-24',
 'peRatio': 13.313656822765799,
 'beta': 1.4312399817947916,
 'maxChangePercent': 58.03887745638058,
 'year5ChangePercent': 4.833038943973622,
 'year2ChangePercent': 1.3642672538156222,
 'year1ChangePercent': 0.28980201788426946,
 'ytdChangePercent': 0.12753760212259269,
 'month6ChangePercent': 0.17183833183732758,
 'month3ChangePercent': 0.018106289406119,
 'month1ChangePercent': 0.03846188001706879,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.28980201788426946

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [43]:
final_dataframe = pd.DataFrame(columns = my_columns)
for batch in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in batch.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                    
                ],
                index = my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,163.500,0.459864,N/A
1,AAL,22.270,0.744753,N/A
2,AAP,244.640,0.550089,N/A
3,AAPL,153.190,0.285158,N/A
4,ABBV,121.800,0.248609,N/A
...,...,...,...,...
500,YUM,126.300,0.246519,N/A
501,ZBH,135.812,-0.103608,N/A
502,ZBRA,600.790,0.687954,N/A
503,ZION,68.570,0.744202,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [44]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.470,2.79351,N/A
1,317,MRO,17.410,2.34436,N/A
2,275,LB,82.730,2.29729,N/A
3,175,FANG,113.210,2.28759,N/A
4,195,FTNT,345.820,2.01955,N/A
5,299,MCHP,84.270,1.74449,N/A
6,357,OXY,32.720,1.64676,N/A
7,111,COTY,11.059,1.46391,N/A
8,345,NVDA,312.640,1.36682,N/A
9,160,EOG,98.930,1.34457,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [45]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')

    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print('That is not a number. \nPlease try again and enter a number.')
        portfolio_size = input('Enter the size of your portfolio')
        portfolio_size = float(portfolio_size)
portfolio_input()

In [46]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.470,2.79351,4600
1,317,MRO,17.410,2.34436,11487
2,275,LB,82.730,2.29729,2417
3,175,FANG,113.210,2.28759,1766
4,195,FTNT,345.820,2.01955,578
5,299,MCHP,84.270,1.74449,2373
6,357,OXY,32.720,1.64676,6112
7,111,COTY,11.059,1.46391,18084
8,345,NVDA,312.640,1.36682,639
9,160,EOG,98.930,1.34457,2021


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'One-Year Price Return',
    'One-Year Return Percentile', 
    'Six-Month Price Return',
    'Six-Month Return Percentile', 
    'Three-Month Price Return',
    'Three-Month Return Percentile', 
    'One-Month Price Return',
    'One-Month Return Percentile', 
]

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: